<a href="https://colab.research.google.com/github/gitcahyani/PortfolioProjects/blob/main/Python_Ecommerce_FinPro_Nila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `Pastikan klik 'File > Save a copy in Drive' sebelum memulai pengerjaan`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay

#Dataset

Data yang digunakan adalah data yang berasal dari Tokopedia (***bukan data sesungguhnya***). Mengenai penjelasan dataset adalah sebagai berikut:

|variable                       |class     |description |
|:------------------------------|:---------|:-----------|
**order_detail:**
id 			|object| angka unik dari order / id_order
customer_id 		|object|angka unik dari pelanggan
order_date 		|object| tanggal saat dilakukan transaksi
sku_id 			|object| angka unik dari produk (sku adalah stock keeping unit)
price			|int64| harga yang tertera pada tagging harga
qty_ordered 		|int64| jumlah barang yang dibeli oleh pelanggan
before_discount	|float64| nilai harga total dari produk (price * qty_ordered)
discount_amount	|float64| nilai diskon product total
after_discount		|float64| nilai harga total produk ketika sudah dikurangi dengan diskon
is_gross 		|int64| menunjukkan pelanggan belum membayar pesanan
is_valid		|int64| menunjukkan pelanggan sudah melakukan pembayaran
is_net			|int64| menunjukkan transaksi sudah selesai
payment_id 		|int64| angka unik dari metode pembayaran
||
**sku_detail:**
id |object| angka unik dari produk (dapat digunakan untuk key saat join)
sku_name 		|object| nama dari produk
base_price		|float64| harga barang yang tertera pada tagging harga / price
cogs 			|int64| cost of goods sold / total biaya untuk menjual 1 produk
category		|object| kategori produk
||
**customer_detail:**
id 			|object| angka unik dari pelanggan
registered_date	|object| tanggal pelanggan mulai mendaftarkan diri sebagai anggota
||
**payment_detail:**
id			|int64| angka unik dari metode pembayaran
payment_method	|object| metode pembayaran yang digunakan


In [ ]:
#Sumber data yang digunakan
path_od = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/order_detail.csv"
path_pd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/payment_detail.csv"
path_cd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/customer_detail.csv"
path_sd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/sku_detail.csv"
df_od = pd.read_csv(path_od)
df_pd = pd.read_csv(path_pd)
df_cd = pd.read_csv(path_cd)
df_sd = pd.read_csv(path_sd)

In [ ]:
#Mengampilkan 5 baris pertama
df_od.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4


In [ ]:
#Mengampilkan 5 baris pertama
df_pd.head()

,id,payment_method
0,1,cod
1,2,jazzvoucher
2,3,customercredit
3,4,Payaxis
4,5,jazzwallet


In [ ]:
#Mengampilkan 5 baris pertama
df_cd.head()

,id,registered_date
0,C996508L,2021-07-10
1,C180415L,2021-07-18
2,C535451L,2021-07-23
3,C177843L,2021-07-12
4,C951682L,2021-07-27


In [ ]:
#Mengampilkan 5 baris pertama
df_sd.head()

,id,sku_name,base_price,cogs,category
0,P798444,AT-FSM-35,57631.70,46052,Kids & Baby
1,P938347,AYS_Haier-18HNF,3931789.26,3499256,Appliances
2,P826364,Atalian _DV206A-Brown-41,324597.00,243426,Men Fashion
3,P467533,Darul_Sakoon_Food_Bundle,2870.42,2378,Superstore
4,P229955,HP_15AY-15-Ay072NIA-ci3,2265625.00,1631250,Computing


In [ ]:
#Menjalankan SQL di Colab
from sqlite3 import connect
conn = connect(':memory:')
df_od.to_sql('order_detail',conn, index=False, if_exists='replace')
df_pd.to_sql('payment_detail', conn, index=False, if_exists='replace')
df_sd.to_sql('sku_detail', conn, index=False, if_exists='replace')
df_cd.to_sql('customer_detail', conn, index=False, if_exists='replace')

3998

In [ ]:
#Query SQL untuk menggabungkan data
df = pd.read_sql("""
SELECT
    order_detail.*,
    payment_detail.payment_method,
    sku_detail.sku_name,
    sku_detail.base_price,
    sku_detail.cogs,
    sku_detail.category,
    customer_detail.registered_date
FROM order_detail
LEFT JOIN payment_detail
    on payment_detail.id = order_detail.payment_id
LEFT JOIN sku_detail
    on sku_detail.id = order_detail.sku_id
LEFT JOIN customer_detail
    on customer_detail.id = order_detail.customer_id
""", conn)

In [ ]:
#Mengampilkan 5 baris pertama
df.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id,payment_method,sku_name,base_price,cogs,category,registered_date
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5,jazzwallet,RB_Dettol Germ Busting Kit-bf,26100.0,18270,Others,2021-07-07
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5,jazzwallet,PS4_Slim-500GB,1971942.0,1321182,Entertainment,2021-11-20
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4,Payaxis,Changhong Ruba 55 Inches UD55D6000i Ultra HD T...,7482000.0,5162580,Entertainment,2021-11-19
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5,jazzwallet,dawlance_Inverter 30,3593680.0,3054628,Appliances,2021-11-03
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4,Payaxis,Dawlance_Inverter-45 2.0 ton,4413220.0,3177472,Appliances,2021-07-05


In [ ]:
#Menampikan tipe data tiap kolom
df.dtypes

id                  object
customer_id         object
order_date          object
sku_id              object
price                int64
qty_ordered          int64
before_discount    float64
discount_amount    float64
after_discount     float64
is_gross             int64
is_valid             int64
is_net               int64
payment_id           int64
payment_method      object
sku_name            object
base_price         float64
cogs                 int64
category            object
registered_date     object
dtype: object

In [ ]:
#Mengubah tipe data agar mudah dilakukan pengolahan data
df = df.astype({"before_discount":'int', "discount_amount":'int', "after_discount":'int',"base_price":'int'})
df.dtypes

id                 object
customer_id        object
order_date         object
sku_id             object
price               int64
qty_ordered         int64
before_discount     int64
discount_amount     int64
after_discount      int64
is_gross            int64
is_valid            int64
is_net              int64
payment_id          int64
payment_method     object
sku_name           object
base_price          int64
cogs                int64
category           object
registered_date    object
dtype: object

In [ ]:
#Mengubah tipe kolom Date menjadi Datetime
df['order_date']= pd.to_datetime(df['order_date'])
df['registered_date']= pd.to_datetime(df['registered_date'])
df.dtypes


id                         object
customer_id                object
order_date         datetime64[ns]
sku_id                     object
price                       int64
qty_ordered                 int64
before_discount             int64
discount_amount             int64
after_discount              int64
is_gross                    int64
is_valid                    int64
is_net                      int64
payment_id                  int64
payment_method             object
sku_name                   object
base_price                  int64
cogs                        int64
category                   object
registered_date    datetime64[ns]
dtype: object

## `No 1`
**Dear Data Analyst**,
<br></br>
Akhir tahun ini, perusahaan akan memberikan hadiah bagi pelanggan yang memenangkan kompetisi **Festival Akhir Tahun**. Tim Marketing membutuhkan bantuan untuk menentukan perkiraan hadiah yang akan diberikan pada pemenang kompetisi nantinya. Hadiah tersebut akan diambil dari **TOP 5 Produk** dari Kategori **Mobiles & Tablets** selama tahun 2022, dengan jumlah kuantitas penjualan (valid = 1) paling tinggi.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data dengan Category = Mobiles & Tablets
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name
5. Gunakan sort_values untuk mengurutkan data
6. Gunakan head untuk menampilkan top 5 produk


```



</details>

## `Jawaban No 1`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
is_valid = df['is_valid'] == 1
category = df['category'] == 'Mobiles & Tablets'
year = df['order_date'].dt.year == 2022
df_filtered = df[(is_valid) & (category) & (year)]
#df_clean = df_filtered[['sku_name', 'qty_ordered']]
df_clean = df_filtered.groupby(by = ['sku_name'])['qty_ordered'].sum().sort_values(ascending=False).head().reset_index(name='qty_2022')
df_clean

,sku_name,qty_2022
0,IDROID_BALRX7-Gold,1000
1,IDROID_BALRX7-Jet black,31
2,Infinix Hot 4-Gold,15
3,samsung_Grand Prime Plus-Black,11
4,infinix_Zero 4-Grey,10


## `No 2`
**Dear Data Analyst**,
<br></br>
Menindaklanjuti meeting gabungan Tim Werehouse dan Tim Marketing, kami menemukan bahwa ketersediaan stock produk dengan Kategori Others pada akhir 2022 kemarin masih banyak.
1. Kami mohon bantuan untuk melakukan pengecekan data penjualan kategori tersebut dengan tahun 2021 secara kuantitas penjualan. Dugaan sementara kami, telah terjadi penurunan kuantitas penjualan pada 2022 dibandingkan 2021. (Mohon juga menampilkan data ke-15 kategori)
<br></br>
2. Apabila memang terjadi penurunan kuantitas penjualan pada kategori Others, kami mohon bantuan untuk menyediakan data TOP 20 nama produk yang mengalami penurunan paling tinggi pada 2022 jika dibanding dengan 2021. Hal ini kami gunakan sebagai bahan diskusi pada meeting selanjutnya.  

Mohon bantuan untuk mengirimkan data tersebut paling lambat 4 hari dari hari ini. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Werehouse**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan valid = 1
2. Memfilter data pada transaksi selama 2021
3. Memfilter data pada transaksi selama 2022
4. Gunakan groupby berdasarkan sku_name masing-masing tahun
5. Menggabungkan kedua data dengan merge
6. Lakukan pengurangan kolom qty 2022 dengan qty 2021


```



</details>

## `Jawaban No 2.1`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
is_valid = df['is_valid'] == 1
category = df['category'] == 'Others'
year2022 = df['order_date'].dt.year == 2022
year2021 = df['order_date'].dt.year == 2021
df_filtered2022 = df[(is_valid) & (year2022)]
df_2022 = df_filtered2022[['category', 'qty_ordered']].groupby(['category'])['qty_ordered'].sum()
df_2022 = df_2022.sort_values(ascending = False).reset_index(name='qty_2022')
#df_2022['year'] = 2022
df_2022

,category,qty_2022
0,Mobiles & Tablets,1154
1,Soghaat,612
2,Superstore,536
3,Women Fashion,489
4,Others,263
5,Home & Living,250
6,School & Education,237
7,Kids & Baby,227
8,Health & Sports,200
9,Books,195


In [ ]:
df_filtered2021 = df[(is_valid) & (year2021)]
df_2021 = df_filtered2021[['category', 'qty_ordered']].groupby('category').sum()
df_2021 = df_2021.sort_values(by = 'qty_ordered', ascending = False)
#df_2021['year'] = 2021
df_2021

,qty_ordered
category,
Soghaat,759
Others,426
Superstore,327
Men Fashion,237
Home & Living,193
School & Education,184
Health & Sports,173
Books,171
Kids & Baby,170


In [ ]:
df_outer = df_2022.merge(df_2021,
                         on = 'category',
                         how = 'outer')
df_outer['growth'] = (df_outer['qty_2022'] - df_outer['qty_ordered'])/ df_outer['qty_2022'] * 100.0
df_outer['remarks'] = np.where(df_outer['growth'] < 0, 'negative growth', 'positive growth')
df_outer.sort_values(by='growth', ascending = True, inplace = True)
df_outer = df_outer.rename(columns = {'qty_2022':'qty_ordered_2022', 'qty_ordered':'qty_ordered_2021'})
df_outer

,category,qty_ordered_2022,qty_ordered_2021,growth,remarks
4,Others,263,426,-61.977186,negative growth
10,Men Fashion,175,237,-35.428571,negative growth
1,Soghaat,612,759,-24.019608,negative growth
11,Beauty & Grooming,153,168,-9.803922,negative growth
9,Books,195,171,12.307692,positive growth
8,Health & Sports,200,173,13.500000,positive growth
14,Appliances,148,124,16.216216,positive growth
6,School & Education,237,184,22.362869,positive growth
5,Home & Living,250,193,22.800000,positive growth
7,Kids & Baby,227,170,25.110132,positive growth


## `Jawaban No 2.2`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
is_valid = df['is_valid'] == 1
category = df['category'] == 'Others'
year2022 = df['order_date'].dt.year == 2022

df_sku_2022 = df[(is_valid) & (category) & (year2022)]
df_sku_2022 = df_sku_2022[['sku_name', 'qty_ordered']].groupby('sku_name').sum()
#df_sku_2022 = df_sku_2022[['sku_name', 'qty_ordered']].groupby(by=["sku_name"])["qty_ordered"].sum().reset_index(name='qty_2022')
df_sku_2022 = df_sku_2022.sort_values(by = 'qty_ordered', ascending = False)
df_sku_2022.head()


,qty_ordered
sku_name,
RB_Dettol Germ Busting Kit-bf,45
emart_Tyre Shape Air Compressor,34
SKMT_Medicine,20
shoppers stop_MagicGlass-3,13
TCF_Educate-a-Child-for-a-Month,8


In [ ]:
df_sku_2021 = df[(is_valid) & (category) & (year2021)]
df_sku_2021 = df_sku_2021[['sku_name', 'qty_ordered']].groupby('sku_name').sum()
df_sku_2021 = df_sku_2021.sort_values(by = 'qty_ordered', ascending = False)
df_sku_2021

,qty_ordered
sku_name,
RB_Dettol Germ Busting Kit-bf,200
Telemall_MM-DR-HB-L,23
Dawlance_MD 10 + DWB 600,23
iu_Tickets General Enclosure-Islamabad,20
RS_Rehmat-e-Shereen Mix Mithai,13
...,...
MEGUIAR_G13115,1
MEGUIAR_G12718,1
MEGUIAR_G10307,1


In [ ]:
df_sku_outer = df_sku_2022.merge(df_sku_2021,
                                 on = 'sku_name',
                                 how = 'outer')
df_sku_outer.fillna(0,inplace=True)
df_sku_outer['difference']= df_sku_outer['qty_ordered_x'] - df_sku_outer['qty_ordered_y']
df_sku_outer = df_sku_outer.sort_values(by = 'difference', ascending = True)
df_sku_outer = df_sku_outer.rename(columns = {'qty_ordered_x':'qty_ordered_2022', 'qty_ordered_y':'qty_ordered_2021'})
df_sku_outer.head(20)

,qty_ordered_2022,qty_ordered_2021,difference
sku_name,,,
RB_Dettol Germ Busting Kit-bf,45.0,200.0,-155.0
Dawlance_MD 10 + DWB 600,0.0,23.0,-23.0
Telemall_MM-DR-HB-L,2.0,23.0,-21.0
iu_Tickets General Enclosure-Islamabad,0.0,20.0,-20.0
RS_Rehmat-e-Shereen Mix Mithai,0.0,13.0,-13.0
kansai_NeverWet,1.0,10.0,-9.0
sindbad_Sindbad Gift Card-3,0.0,7.0,-7.0
emart_00-1,1.0,7.0,-6.0
Vouch 365 2016,0.0,5.0,-5.0


## `No 3`
**Dear Data Analyst**,
<br></br>
Terkait ulang tahun perusahaan pada 2 bulan mendatang, Tim Digital Marketing akan memberikan informasi promo bagi pelanggan pada akhir bulan ini. Kriteria pelanggan yang akan kami butuhkan adalah mereka yang sudah melakukan check-out namun belum melakukan pembayaran (is_gross = 1) selama tahun 2022. Data yang kami butuhkan adalah ID Customer dan Registered Date.

Mohon bantuan, untuk mengirimkan data tersebut sebelum akhir bulan ini ke Tim Digital Marketing. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Digital Marketing**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Memfilter data dengan gross = 1
2. Memfilter data dengan valid = 0
3. Memfilter data dengan net = 0
4. Memfilter data pada transaksi selama 2022


```



</details>

## `Jawaban No 3`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
is_gross = df['is_gross'] == 1
is_valid = df['is_valid'] == 0
is_net = df['is_net'] == 0
is_2022 = df['order_date'].dt.year == 2022
df_filtered_cust = df[(is_gross) & (is_valid) & (is_net) & (is_2022)]
df_filtered_cust = df_filtered_cust[['customer_id', 'registered_date']]
df_filtered_cust.drop_duplicates()

,customer_id,registered_date
9,C246762L,2022-05-08
18,C848774L,2021-11-07
19,C693415L,2022-04-12
21,C180595L,2022-04-22
22,C587425L,2022-03-22
...,...,...
5855,C653797L,2022-04-03
5856,C394076L,2021-10-12
5859,C248585L,2022-07-10
5865,C471304L,2022-05-13


In [ ]:
#Jalankan kode ini untuk mendownload file
from google.colab import files
df_filtered_cust.to_csv('audience_list.csv', encoding = 'utf-8-sig',index=False) #ganti [nama variabel file] dengan nama variabel yang digunakan
files.download('audience_list.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## `No 4`
**Dear Data Analyst**,
<br></br>
Pada bulan October hingga Desember 2022, kami melakukan campaign setiap hari Sabtu dan Minggu. Kami hendak menilai, apakah campaign tersebut cukup berdampak pada kenaikan penjualan (before_discount). Mohon bantuan untuk menampilkan data:
1. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) per bulan tersebut. Apakah ada peningkatan penjualan pada masing-masing bulan tersebut.
2. Rata-rata harian penjualan weekends (Sabtu dan Minggu) vs rata-rata harian penjualan weekdays (Senin-Jumat) keseluruhan 3 bulan tersebut.

Mohon bantuan untuk mengirimkan data tersebut paling lambat minggu depan. Atas bantuan yang diberikan, kami mengucapkan terima kasih.
<br></br>
Regards

**Tim Campaign**

<details><summary>Klik di sini untuk tips</summary>


```python
1. Buatlah kolom baru untuk day, month, dan month number
2. Memfilter data dengan valid = 1
3. Memfilter data dengan day termasuk 'Saturday', 'Sunday' dengan menggunakan fungsi isin dan simpan dalam variable 'weekends'
4. Memfilter data dengan day termasuk 'Monday','Tuesday','Wednesday','Thusday','Friday' dengan menggunakan fungsi isin dan simpan dengan variabel 'weekday'
5. Memfilter data pada transaksi selama 2022 bulan Oktober sampai dengan Desember
6. Gunakan groupby berdasarkan pada masing-masing variabel
7. Menggabungkan kedua data dengan merge
8. Tampilkan pada grafik batang untuk melihat perbedaananya


```



</details>

## `Jawaban No 4.1`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
df['day'] = df['order_date'].dt.day_name()
df['month'] = df['order_date'].dt.month
df['month_name'] = df['order_date'].dt.month_name()
filter_valid = df['is_valid'] == 1
filter_Q42022 = (df['order_date'].dt.month.isin({10,11,12})) & (df['order_date'].dt.year == 2022)
weekends = df['day'].isin(['Saturday', 'Sunday'])
#dataframe setelah di-filter
df_weekends = df[(filter_valid) & (filter_Q42022) & (weekends)]
#total penjualan harian weekend
df_weekends = df_weekends[['order_date','month','month_name', 'before_discount']].groupby(by=['order_date','month','month_name']).sum()
avg_weekends = df_weekends.groupby(by=['month', 'month_name'])['before_discount'].mean().round().reset_index(name='avg_sales_weekend')
avg_weekends

,month,month_name,avg_sales_weekend
0,10,October,5708341.0
1,11,November,5774045.0
2,12,December,4105994.0


In [ ]:
weekdays = df['day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
df_weekdays = df[(filter_valid) & (filter_Q42022) & (weekdays)]
df_weekdays = df_weekdays[['order_date', 'month_name', 'month', 'before_discount']].groupby(by=['order_date','month_name', 'month']).sum()
avg_weekdays = df_weekdays.groupby(by=['month', 'month_name'])['before_discount'].mean().round().reset_index(name='avg_sales_weekdays')
avg_weekdays

,month,month_name,avg_sales_weekdays
0,10,October,7872212.0
1,11,November,6204666.0
2,12,December,8542530.0


In [ ]:
df_merge = avg_weekends.merge(avg_weekdays,
                             on = 'month')
df_merge.sort_values(by = 'month', ascending=True, inplace = True)
df_merge = df_merge[['month', 'avg_sales_weekend', 'avg_sales_weekdays']]
df_merge['diff_value'] = df_merge['avg_sales_weekend'] - df_merge['avg_sales_weekdays']
df_merge['diff_perc'] = round(df_merge['diff_value']/df_merge['avg_sales_weekend']*100.0)
df_merge

,month,avg_sales_weekend,avg_sales_weekdays,diff_value,diff_perc
0,10,5708341.0,7872212.0,-2163871.0,-38.0
1,11,5774045.0,6204666.0,-430621.0,-7.0
2,12,4105994.0,8542530.0,-4436536.0,-108.0


## `Jawaban No 4.2`

In [ ]:
# Tulis kode Anda di bawah ini. Dapat menggunakan lebih dari 1 blok kode
df_q4 = {'Period' : 'Q4 2022',\
         'Avg Weekend Sales': round(df_weekends['before_discount'].mean(),2),\
         'Avg Weekdays Sales' : round(df_weekdays['before_discount'].mean(),2),\
         'Diff (Value)' : round(df_weekends['before_discount'].mean() - df_weekdays['before_discount'].mean(),2),\
         'Diff (%)' : round((df_weekends['before_discount'].mean() - df_weekdays['before_discount'].mean())/df_weekends['before_discount'].mean() ,2)*100.0
}

pd.DataFrame(data=df_q4, index=[0])

,Period,Avg Weekend Sales,Avg Weekdays Sales,Diff (Value),Diff (%)
0,Q4 2022,5269300.0,7520249.29,-2250949.29,-43.0


In [ ]:
data21 = {\
          'Periode':'Total 3 months',\
          'Avg Weekend Sales': round(daily_weekend['avg_sales_weekend'].mean(),2), \
          'Avg Weekdays Sales': round(daily_weekday['avg_sales_weekday'].mean(),2),\
          'Diff (Value)': round(daily_weekend['avg_sales_weekend'].mean() - daily_weekday['avg_sales_weekday'].mean(),2),\
          'Diff (%)': pd.Series(round(((daily_weekend['avg_sales_weekend'].mean() - daily_weekday['avg_sales_weekday'].mean())/daily_weekend['avg_sales_weekend'].mean())*100,2), dtype=str)+'%'
          }
pd.DataFrame(data=data21, index=[0])